## Parameters

In [42]:
input_data='./input_data/'
output_data='./output_data/'

## Libraries

In [43]:
import numpy as np
import pandas as pd
import datetime

In [44]:
cc_case='WC1'

Load the worst case impact table. 

WC1 and WC2 are the two worst case impact predictions giving reduction in flow by month for 2070.
- *WC1*: Hamududu & Killingtviet 2016
- *WC2*: Harrison & Whittington, 2002

In [45]:
cc_cases=pd.read_csv(input_data + 'climate_change_worst_cases.csv').set_index('Month')
cc_cases

,WC1,WC2
Month,,
10,-0.16,-0.243920
11,-0.20,-0.296011
12,-0.17,-0.396367
1,-0.18,-0.326546
2,-0.16,-0.324661
3,-0.41,-0.395765
4,-0.29,-0.386358
5,-0.24,-0.362866
6,-0.24,-0.350233


In [46]:
daily_flow=pd.read_csv(input_data + 'ngonye_flow_mmlv2.csv') #output_data + 'ngonye_flow_daily.csv')

daily_flow=daily_flow.set_index(pd.DatetimeIndex(daily_flow['Date']))
if 'LaggedDate' in daily_flow.columns:
    daily_flow=daily_flow.drop(['LaggedDate','VicFalls','Conversion','Flow_difference','Volume','MonthId','Date','WaterMonth','WaterDay','WaterWeek','Exceedance'],axis=1)
else:
    daily_flow=daily_flow.drop(['Date'],axis=1)
    
    daily_flow['Day']=daily_flow.index.day
    daily_flow['Month']=daily_flow.index.month
    daily_flow['Year']=daily_flow.index.year
daily_flow.head(4)

,Flow,Day,Month,Year
Date,,,,
1924-10-01,111.4075,1,10,1924
1924-10-02,111.4075,2,10,1924
1924-10-03,111.4075,3,10,1924
1924-10-04,111.4075,4,10,1924


In [47]:
sims=pd.DataFrame(np.arange(1,61,1),columns=['SimNo'])
sims['StartDate']=pd.date_range(start=datetime.date(1933, 10, 1), periods=60, freq=pd.tseries.offsets.DateOffset(years=1))
sims=sims.set_index('SimNo')
sims

,StartDate
SimNo,
1,1933-10-01
2,1934-10-01
3,1935-10-01
4,1936-10-01
5,1937-10-01
6,1938-10-01
7,1939-10-01
8,1940-10-01
9,1941-10-01


Calculate the Climate Change Rate which is 0% in 1990 and rises to 100% in 2070. This sets the proportion of the overall climate change impact (by 2070) which is assumed to apply to any given date.

In [48]:
daily_flow['CC_Factor']=daily_flow.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
daily_flow['Impact_2070']=daily_flow.join(cc_cases,on='Month')[cc_case]
daily_flow['Flow_Adjustment']=(1+daily_flow['CC_Factor']*(-daily_flow['Impact_2070']))
daily_flow['Flow_Adjusted']=daily_flow['Flow']*daily_flow['Flow_Adjustment']
daily_flow

,Flow,Day,Month,Year,CC_Factor,Impact_2070,Flow_Adjustment,Flow_Adjusted
Date,,,,,,,,
1924-10-01,111.4075,1,10,1924,0.0000,-0.16,1.00000,111.407500
1924-10-02,111.4075,2,10,1924,0.0000,-0.16,1.00000,111.407500
1924-10-03,111.4075,3,10,1924,0.0000,-0.16,1.00000,111.407500
1924-10-04,111.4075,4,10,1924,0.0000,-0.16,1.00000,111.407500
1924-10-05,111.4075,5,10,1924,0.0000,-0.16,1.00000,111.407500
...,...,...,...,...,...,...,...,...
2017-09-26,200.6671,26,9,2017,0.3375,-0.14,1.04725,210.148620
2017-09-27,200.6671,27,9,2017,0.3375,-0.14,1.04725,210.148620
2017-09-28,200.6671,28,9,2017,0.3375,-0.14,1.04725,210.148620


In [50]:
tmp=daily_flow.reset_index().head(4018)[['Date','Flow','Flow_Adjusted']].copy()
tmp['Date']=pd.date_range(start=datetime.date(2017, 10, 1), periods=4018, freq='D')
tmp=tmp.set_index('Date')
tmp['Day']=tmp.index.day
tmp['Month']=tmp.index.month
tmp['Year']=tmp.index.year
tmp['WaterYear']=tmp.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
tmp

,Flow,Flow_Adjusted,Day,Month,Year,WaterYear
Date,,,,,,
2017-10-01,111.4075,111.4075,1,10,2017,2017.0
2017-10-02,111.4075,111.4075,2,10,2017,2017.0
2017-10-03,111.4075,111.4075,3,10,2017,2017.0
2017-10-04,111.4075,111.4075,4,10,2017,2017.0
2017-10-05,111.4075,111.4075,5,10,2017,2017.0
...,...,...,...,...,...,...
2028-09-26,258.7976,258.7976,26,9,2028,2027.0
2028-09-27,258.7976,258.7976,27,9,2028,2027.0
2028-09-28,256.6828,256.6828,28,9,2028,2027.0


In [51]:
daily=daily_flow.append(tmp)
daily=daily.drop(['CC_Factor','Impact_2070','Flow_Adjustment'],axis=1)
daily

,Flow,Day,Month,Year,Flow_Adjusted,WaterYear
Date,,,,,,
1924-10-01,111.4075,1,10,1924,111.4075,NaN
1924-10-02,111.4075,2,10,1924,111.4075,NaN
1924-10-03,111.4075,3,10,1924,111.4075,NaN
1924-10-04,111.4075,4,10,1924,111.4075,NaN
1924-10-05,111.4075,5,10,1924,111.4075,NaN
...,...,...,...,...,...,...
2028-09-26,258.7976,26,9,2028,258.7976,2027.0
2028-09-27,258.7976,27,9,2028,258.7976,2027.0
2028-09-28,256.6828,28,9,2028,256.6828,2027.0


In [52]:
future=pd.DataFrame(index=pd.date_range(start='10/1/2017', end='09/30/2053', freq='D'))
future['Year']=future.index.year
future['Month']=future.index.month
future['CC_Factor']=future.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
future

,Year,Month,CC_Factor
2017-10-01,2017,10,0.3375
2017-10-02,2017,10,0.3375
2017-10-03,2017,10,0.3375
2017-10-04,2017,10,0.3375
2017-10-05,2017,10,0.3375
...,...,...,...
2053-09-26,2053,9,0.7875
2053-09-27,2053,9,0.7875
2053-09-28,2053,9,0.7875
2053-09-29,2053,9,0.7875


In [53]:

def calc_flows(sim_no):
    start_date=sims.loc[sim_no]['StartDate']

    future['SampleDate']=pd.date_range(start=start_date, periods=13149, freq='D')
    future['SampleFlow']=future.join(daily,on='SampleDate',rsuffix='_daily')['Flow_Adjusted']
    future['Impact_2070']=future.join(cc_cases,on='Month')[cc_case]
    future['Flow' + str(sim_no)]=future['SampleFlow']*(1+future['Impact_2070']*future['CC_Factor'])
 

In [54]:
for sim in sims.itertuples():
    calc_flows(sim.Index)
    
future

,Year,Month,CC_Factor,SampleDate,SampleFlow,Impact_2070,Flow1,Flow2,Flow3,Flow4,...,Flow51,Flow52,Flow53,Flow54,Flow55,Flow56,Flow57,Flow58,Flow59,Flow60
2017-10-01,2017,10,0.3375,1992-10-01,175.743975,-0.16,178.529025,222.090433,242.821929,197.973677,...,208.621853,200.600624,214.147912,244.822530,226.700670,232.900943,258.981122,212.538861,188.718283,166.253801
2017-10-02,2017,10,0.3375,1992-10-02,171.822552,-0.16,178.529025,218.932307,242.821929,195.440384,...,208.621853,200.600624,214.147912,242.821929,226.700670,232.900943,258.981122,211.459853,188.718283,162.544134
2017-10-03,2017,10,0.3375,1992-10-03,171.822552,-0.16,178.529025,214.147912,236.310138,195.440384,...,208.621853,197.973677,212.538861,232.900943,226.700670,232.900943,256.623785,208.210059,188.718283,162.544134
2017-10-04,2017,10,0.3375,1992-10-04,171.822552,-0.16,175.746650,212.538861,232.900943,195.440384,...,203.156527,197.973677,212.538861,232.900943,226.700670,230.142596,256.623785,208.621853,188.718283,162.544134
2017-10-05,2017,10,0.3375,1992-10-05,171.822552,-0.16,175.746650,210.279529,226.700670,195.440384,...,203.156527,197.973677,210.279529,230.142596,226.700670,230.142596,256.623785,208.621853,188.718283,162.544134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053-09-26,2053,9,0.7875,2028-09-26,258.797600,-0.14,387.483367,303.044668,295.051332,266.195583,...,191.076660,173.319385,232.565435,178.170658,237.372844,237.372844,237.372844,171.628593,219.052446,230.265165
2053-09-27,2053,9,0.7875,2028-09-27,258.797600,-0.14,383.444703,303.044668,292.910772,266.195583,...,186.201986,171.628593,230.265165,173.686051,237.372844,228.383521,230.265165,171.966164,213.220847,230.265165
2053-09-28,2053,9,0.7875,2028-09-28,256.682800,-0.14,383.444703,301.021199,289.756697,268.975696,...,183.819325,171.966164,230.265165,173.686051,237.372844,222.258927,230.265165,168.375934,210.153878,228.383521
2053-09-29,2053,9,0.7875,2028-09-29,256.682800,-0.14,379.596534,301.021199,287.946678,268.975696,...,178.991096,171.966164,228.383521,173.686051,232.565435,222.258927,228.383521,168.375934,210.153878,228.383521


In [73]:

for sim in sims.itertuples():
    sims.at[sim.Index,'Mean_2023_2040']=future.loc[(future.index>=datetime.datetime(2023, 10, 1)) & (future.index<=datetime.datetime(2041, 9, 30))]['Flow' + str(sim.Index)].mean()
    sims.at[sim.Index,'Mean_2023_2052']=future.loc[(future.index>=datetime.datetime(2023, 10, 1)) & (future.index<=datetime.datetime(2053, 9, 30))]['Flow' + str(sim.Index)].mean()

sims

,StartDate,Mean_2023_2040,Mean_2023_2052
SimNo,,,
1,1933-10-01,995.262046,1084.154211
2,1934-10-01,1040.549033,1089.245594
3,1935-10-01,1048.353392,1093.802730
4,1936-10-01,1072.257653,1099.729374
5,1937-10-01,1111.183816,1099.807441
6,1938-10-01,1149.005962,1106.214709
7,1939-10-01,1201.152905,1123.231296
8,1940-10-01,1219.226414,1148.152911
9,1941-10-01,1207.718542,1146.320443


In [ ]:
summary